* CycleGANを用いてピラフからカレーライスへの画像変換を行う


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# カレントディレクトリの読み込みとカレントディレクトリへの移動
import sys
sys.path.append(f'/content/drive/My Drive/system/')
import os
os.chdir(f'/content/drive/My Drive/system/myanswer')

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-ln3kk7gc
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-ln3kk7gc
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=f8452e893b9ffe806457e01d19e1d46dc87748e7ee166fffaef17d18d669a340
  Stored in directory: /tmp/pip-ephem-wheel-cache-6om21y48/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [ ]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 100kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
from __future__ import print_function, division

from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from glob import glob
from PIL import Image
import datetime
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.misc

In [ ]:
class DataGenerator():
      """
      source_nameで指定した画像(A)を target_nameで指定した画像(B)に変換する
      そのために，画像を収集し，AとBの画像を学習データとテストデータに分け
      それぞれ，学習データ用とテストデータ用のディレクトリに保存するクラス
      """
      def __init__(self, source_name, target_name, dataset_name, img_res=(128, 128)):
        self.source_name = source_name
        self.target_name = target_name
        self.dataset_name = dataset_name 
        self.img_res = img_res

      def generate_data(self, indices=1000, train_percentage=0.8):
         """
          画像の収集とCycleGANを実行するための画像パスを設定し，収集した画像を設定したパスへと保存するメソッド

          indices(int): 画像枚数
          デフォルトでは，self.target_nameで指定した画像のうち
          8割 (train_percentage): 学習データ用の画像
          2割 : テストデータ用の画像
         """
         os.makedirs('../figure/foodimg128/{}/trainA'.format(self.dataset_name), exist_ok=True)
         os.makedirs('../figure/foodimg128/{}/trainB'.format(self.dataset_name), exist_ok=True)
         os.makedirs('../figure/foodimg128/{}/testA'.format(self.dataset_name), exist_ok=True)
         os.makedirs('../figure/foodimg128/{}/testB'.format(self.dataset_name), exist_ok=True)
         
         # self.source_nameで指定された食事画像のパスを取得
         source_image_pathes = glob('../figure/foodimg128/{}/*.jpg'.format(self.source_name))
         # self.target_nameで指定された食事画像のパスを取得
         target_image_pathes = glob('../figure/foodimg128/{}/*.jpg'.format(self.target_name))

         for index in range(indices):
            source_image = Image.open(source_image_pathes[index])
            target_image = Image.open(target_image_pathes[index])
            if indices * train_percentage <= index < indices:
                source_image.save("../figure/foodimg128/{}/testA/{}.jpg".format(self.dataset_name, index)) # テストデータの保存
                target_image.save("../figure/foodimg128/{}/testB/{}.jpg".format(self.dataset_name, index))
            else:
                source_image.save("../figure/foodimg128/{}/trainA/{}.jpg".format(self.dataset_name, index)) # 学習データの保存
                target_image.save("../figure/foodimg128/{}/trainB/{}.jpg".format(self.dataset_name, index))      

In [ ]:
class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('../figure/foodimg128/%s/%s/*.jpg' % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = scipy.misc.imresize(img, self.img_res)
                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = scipy.misc.imresize(img, self.img_res)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "test"
        path_A = glob('../figure/foodimg128/%s/%sA/*.jpg' % (self.dataset_name, data_type))
        path_B = glob('../figure/foodimg128/%s/%sB/*.jpg' % (self.dataset_name, data_type))

        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)
                

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def load_img(self, path):
        img = self.imread(path)
        img = scipy.misc.imresize(img, self.img_res)
        img = img/127.5 - 1.
        return img[np.newaxis, :, :, :]

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)

In [ ]:
class CycleGAN():
    def __init__(self):
        # Number of images (簡単のため，入力に使う画像枚数を既知であることが前提)
        self.image_num = 2000
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        # ピラフからカレーへの画像変換を行う
        self.dataset_name = 'pilaf2curry'
        self.source_name = "pilaf"
        self.target_name = "curry"
        
        self.data_generator = DataGenerator(source_name=self.source_name,
                                            target_name=self.target_name, 
                                            dataset_name=self.dataset_name, 
                                            img_res=(self.img_rows, self.img_cols))
        
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 32
        self.df = 64

        # Loss weights
        self.lambda_cycle = 10.0                    # Cycle-consistency loss
        self.lambda_id = 0.1 * self.lambda_cycle    # Identity loss

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        # Identity mapping of images
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        reconstr_A, reconstr_B,
                                        img_A_id, img_B_id ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                            loss_weights=[  1, 1,
                                            self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id ],
                            optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)

        # Upsampling
        u1 = deconv2d(d4, d3, self.gf*4)
        u2 = deconv2d(u1, d2, self.gf*2)
        u3 = deconv2d(u2, d1, self.gf)

        u4 = UpSampling2D(size=2)(u3)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def generate_dataset(self):
        partitions = ["trainA", "trainB", "testA", "testB"]
        total_images = 0
          
        for partition in partitions:
            images_each_partition = os.listdir("../figure/foodimg128/{}/{}".format(self.dataset_name, partition))
            total_images += len(images_each_partition)

        if total_images == self.image_num:
            return
        else:
            self.data_generator.generate_data()

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()

        # Generate data for cyclegan
        self.generate_dataset()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):
                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)


                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                      [valid, valid,
                                                       imgs_A, imgs_B,
                                                       imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time

             
                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)
                    # Plot the progress
                    print("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                        % ( epoch+1, epochs,
                                                                          batch_i+1, self.data_loader.n_batches,
                                                                          d_loss[0], 100*d_loss[1],
                                                                          g_loss[0],
                                                                          np.mean(g_loss[1:3]),
                                                                          np.mean(g_loss[3:5]),
                                                                          np.mean(g_loss[5:6]),
                                                                          elapsed_time))
                
                elif batch_i + 2  == self.data_loader.n_batches:
                    self.sample_images(epoch, batch_i)
                    # Plot the progress
                    print("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                        % ( epoch+1, epochs,
                                                                          batch_i+1, self.data_loader.n_batches,
                                                                          d_loss[0], 100*d_loss[1],
                                                                          g_loss[0],
                                                                          np.mean(g_loss[1:3]),
                                                                          np.mean(g_loss[3:5]),
                                                                          np.mean(g_loss[5:6]),
                                                                          elapsed_time))


    def sample_images(self, epoch, batch_i):
        os.makedirs('../result/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
        imgs_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c, figsize=(10, 10))
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("../result/{}/epoch{}_imageid{}.png".format(self.dataset_name, epoch+1, batch_i),
                     transparent=True, dpi=300, bbox_inches="tight", pad_inches=0.0)
        if batch_i + 2 == self.data_loader.n_batches:
            fig.savefig("../result/{}/epoch{}_imageid{}.png".format(self.dataset_name, epoch+1, batch_i+2),
                        transparent=True, dpi=300, bbox_inches="tight", pad_inches=0.0)
        plt.close()

In [ ]:
cyclegan = CycleGAN()
cyclegan.train(epochs=200, batch_size=1, sample_interval=200)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


[Epoch 1/1000] [Batch 1/800] [D loss: 16.744996, acc:  33%] [G loss: 79.088791, adv: 28.967282, recon: 0.968649, id: 1.042343] time: 0:00:34.424808 
[Epoch 1/1000] [Batch 201/800] [D loss: 0.159293, acc:  76%] [G loss: 7.526360, adv: 0.846814, recon: 0.206265, id: 0.574997] time: 0:03:13.819964 
[Epoch 1/1000] [Batch 401/800] [D loss: 0.208001, acc:  60%] [G loss: 5.379257, adv: 0.609793, recon: 0.132221, id: 0.383562] time: 0:05:29.066955 
[Epoch 1/1000] [Batch 601/800] [D loss: 0.263280, acc:  51%] [G loss: 5.735487, adv: 0.597524, recon: 0.136400, id: 0.682430] time: 0:07:43.951798 
[Epoch 1/1000] [Batch 799/800] [D loss: 0.137236, acc:  81%] [G loss: 5.370963, adv: 0.708084, recon: 0.111999, id: 0.682203] time: 0:09:56.322172 
[Epoch 2/1000] [Batch 1/800] [D loss: 0.393398, acc:  39%] [G loss: 6.163262, adv: 0.681187, recon: 0.163701, id: 0.420512] time: 0:09:59.223762 
[Epoch 2/1000] [Batch 201/800] [D loss: 0.200440, acc:  67%] [G loss: 5.350718, adv: 0.661782, recon: 0.119107, i